<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Run_pyserini_distillbert_kd_tasb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distilbert-KD TASB Passage Ranking on MSMARCO

Sebastian Hofstätter, Sheng-Chieh Lin, Jheng-Hong Yang, Jimmy Lin, Allan Hanbury. Efficiently Teaching an Effective Dense Retriever with Balanced Topic Aware Sampling. SIGIR 2021.


## Install dependencies

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
%%capture
!pip install pyserini

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
!pip install faiss-cpu

## Dense retrieval with Distilbert-KD TASB - MSMARCO Passage Ranking

In [ ]:
!$ python -m pyserini.dsearch --topics msmarco-passage-dev-subset \
                             --index msmarco-passage-distilbert-dot-tas_b-b256-bf \
                             --encoded-queries distilbert_tas_b-msmarco-passage-dev-subset \
                             --batch-size 36 \
                             --threads 12 \
                             --output run.msmarco-passage.distilbert-dot-tas_b-b256.bf.tsv \
                             --output-format msmarco

In [ ]:
!gsutil cp run.msmarco-passage.distilbert-dot-tas_b-b256.bf.tsv gs://luanps/information_retrieval/pyserini/distilbert-kd-tasb/

### Evaluation

In [ ]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.msmarco-passage.distilbert-dot-tas_b-b256.bf.tsv >> msmarco_passage_distilbert-dot-tas_b-b256_bf_mrr_eval.txt

In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.msmarco-passage.distilbert-dot-tas_b-b256.bf.tsv \
                                                         --output run.msmarco-passage.distilbert-dot-tas_b-b256.bf.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.msmarco-passage.distilbert-dot-tas_b-b256.bf.trec >> msmarco_passage_distilbert-dot-tas_b-b256.bf.tsv_eval.txt

In [ ]:
!gsutil cp msmarco_passage_distilbert-dot-tas_b-b256_bf_mrr_eval.txt msmarco_passage_distilbert-dot-tas_b-b256.bf.tsv_eval.txt gs://luanps/information_retrieval/pyserini/distilbert-kd-tasb/